In [7]:
!pip uninstall sktime

Found existing installation: sktime 0.27.0
Uninstalling sktime-0.27.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/build_tools/changelog.py
    /usr/local/lib/python3.10/dist-packages/build_tools/make_release.py
    /usr/local/lib/python3.10/dist-packages/docs/Makefile
    /usr/local/lib/python3.10/dist-packages/docs/make.bat
    /usr/local/lib/python3.10/dist-packages/docs/source/_static/css/custom.css
    /usr/local/lib/python3.10/dist-packages/docs/source/_static/css/fields.css
    /usr/local/lib/python3.10/dist-packages/docs/source/_static/js/dynamic_table.js
    /usr/local/lib/python3.10/dist-packages/docs/source/_templates/class.rst
    /usr/local/lib/python3.10/dist-packages/docs/source/_templates/class_with_call.rst
    /usr/local/lib/python3.10/dist-packages/docs/source/_templates/function.rst
    /usr/local/lib/python3.10/dist-packages/docs/source/_templates/layout.html
    /usr/local/lib/python3.10/dist-packages/docs/source/_templates/numpydoc_docstring.rst
 

In [8]:
!pip install sktime

  Using cached sktime-0.27.0-py3-none-any.whl (21.9 MB)


In [3]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sn
import os.path, sys, re
import time
from PIL import Image


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_predict

from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier

from sktime.datasets import load_basic_motions
from sktime.transformations.panel.compose import ColumnConcatenator


In [8]:
#Mount Google Drive
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [9]:
pathroot = "/drive/MyDrive/ECG/CSV_data_v2/"

In [10]:
LeadDict={'Lead1':np.array([[]]),'Lead2':np.array([[]]),'Lead3':np.array([[]]),'Lead4':np.array([[]]),
          'Lead5':np.array([[]]),'Lead6':np.array([[]]),'Lead7':np.array([[]]),'Lead8':np.array([[]]),
         'Lead9':np.array([[]]),'Lead10':np.array([[]]),'Lead11':np.array([[]]),'Lead12':np.array([[]])}

In [11]:
leadMinLen = {'Lead1':0,'Lead2':0,'Lead3':0,'Lead4':0,'Lead5':0,'Lead6':0,
           'Lead7':0,'Lead8':0,'Lead9':0,'Lead10':0,'Lead11':0,'Lead12':0}

In [12]:
ClassLabels={'ECGImagesofPatientthathaveHistoryofMI':0,'ECGImagesofPatientthathaveabnormalheartbeat':1,
            'ECGImagesofCOVID-19Patients':2,'NormalPersonECGImages':3,'ECGImagesofMyocardialInfarctionPatients':4}

In [13]:
"""
For every "x" value of the signal, average the "y" values over duplicates.

Inputs:
-------
df  data frame of two columns containing the signal "x" and "y" coordinates corresponding to "active" pixels

Outputs:
--------
signal numpy array of unique values ("y") of the signal
"""
def uniqValsSignal(df):
    xdf=df[0].to_numpy()
    ydf=df[1].to_numpy()
    unikVals = pd.unique(xdf)
    signal = np.zeros(len(unikVals))
    for i in range(len(unikVals)):
        mask = (xdf==unikVals[i])
        signal[i] = np.mean(ydf[mask])
    return signal

In [14]:
"""
Note:
-----

1) For the "time series", only the second column of every lead is extracted as signal.
2) Because of non-uniformity in signal lenght across both observations and Leads, the code
uses the minimum signal length across observation and leads, in order to make the date "proper
for multivariate time series classification.
"""

labelArr = np.array([])
for dirs in os.listdir(pathroot):
#     if dirs == 'ECGImagesofCOVID-19Patients':
#         continue
    t = time.time()
    count = 0
    print('Processing {0} folder ...'.format(dirs))
    if not os.path.isfile(dirs):
        for item in os.listdir(os.path.join(pathroot,dirs)):
            #print('Processing {0} file ...'.format(item))
            of, oe = os.path.splitext(item)
            if of[0]=='.':
                continue
            else:
                signal = pd.read_csv(os.path.join(pathroot,os.path.join(dirs,item)),header=None,sep=' ')#[1].to_numpy()
                signal = uniqValsSignal(signal)
            try:
                leadNum = int(of[-2:])
            except ValueError:
                leadNum = int(of[-1:])
            finally:
                leadKey = 'Lead'+str(leadNum)
            if leadNum == 13:
                continue
            if LeadDict[leadKey].shape[1] > 0:
                if len(signal)> leadMinLen[leadKey]:
                    signal = np.reshape(signal[:leadMinLen[leadKey]],(1,leadMinLen[leadKey]))
                    LeadDict[leadKey] = np.concatenate( (LeadDict[leadKey],signal) )
                else:
                    LeadDict[leadKey] = LeadDict[leadKey][:,:len(signal)]
                    LeadDict[leadKey] = np.concatenate( (LeadDict[leadKey],np.reshape(signal,(1,len(signal)))) )
                    leadMinLen[leadKey] = len(signal)
            else:
                LeadDict[leadKey] = np.reshape(signal,(1,len(signal)))
                leadMinLen[leadKey] = len(signal)
            count = count+1
        labelArr = np.append(labelArr,np.repeat(ClassLabels[dirs],len(LeadDict[leadKey])-len(labelArr))) ##Add labels
        t = time.time()-t
        print('{0} files processed in this folder in {1} sec...\n'.format(count,round(t)))

Processing ECGImagesofMyocardialInfarctionPatients folder ...


ValueError: invalid literal for int() with base 10: ')'